First set up code to visualize a sound form

In [1]:
%pylab inline
import IPython.display as ipd
import librosa
from librosa import load, display
import glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()
import pickle
from common import save_as_pickle
# import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np

In [3]:
# You should change these paths according to the path of the files on your system.
PATH_TO_TRAIN_LABELS = "data/train/train.csv"
PATH_TO_TEST_LABELS = "data/test/test.csv"
PATH_TO_TRAIN_AUDIO_FILES = "data/train/wav/"
PATH_TO_TEST_AUDIO_FILES = "data/test/wav/"
PATH_TO_SUBMISSION = "submission/"
PATH_TO_PICKLE = "pickles/"
SUBMISSION_TITLE = "nn tonnetz"

In [4]:
# It is easier to deal with csv if you can load it into a structure you can work with.
# Pandas are the most convenient way to do that and are available with 
# inbuilt functionality to handle csv file.

# Pandas assumes that the first row in your file is the header adn not the actual values.
# This behavior can be overriden by passing header=None as a parameter.
train = pd.read_csv(PATH_TO_TRAIN_LABELS)
test = pd.read_csv(PATH_TO_TEST_LABELS)

In [5]:
# You can reactivate this cell to make sure your model is working correctly in terms of dimensions.
#train = train[:2]
#test = test[:2]

In [6]:
train_error_count = 0
train_error_labels = []
test_error_count = 0
test_error_labels = []

In [7]:
# To start with classification, we first need to convert the wav sound files into a format we can work 
# with. It is easier to take the amplitude at each sampling point and use that 
# numeric value to form a feature vector.
def train_parser(row):
    global train_error_count
    global train_error_labels
    path_to_wav_files = PATH_TO_TRAIN_AUDIO_FILES
    file_path = path_to_wav_files + str(row.ID) + ".wav"
    try:
        data, sampling_rate = librosa.load(file_path)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(data),sr=sampling_rate).T,axis=0)
    except Exception as ex:
        print(ex)
        train_error_count += 1
        train_error_labels.append(row.ID)
        return [0]*6, row.Class
    features = tonnetz
    label = row.Class
    return [features, label]

In [8]:
# To create the training feature matrix, we can apply our parser to each training sample.
train_features = train.progress_apply(train_parser,axis=1)
print("%d samples had errors while parsing" % train_error_count)
print("Errorneous samples", train_error_labels)
save_as_pickle(data=train_features,pickle_file=PATH_TO_PICKLE + SUBMISSION_TITLE + " train.pickle" )

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\librosa\util\utils.py:1467: RuntimeWarning: invalid value encountered in less
  if np.any(X < 0) or np.any(X_ref < 0):
C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\librosa\util\utils.py:1480: RuntimeWarning: invalid value encountered in less
  bad_idx = (Z < np.finfo(dtype).tiny)


Audio buffer is not finite everywhere
Audio buffer is not finite everywhere
Audio buffer is not finite everywhere

3 samples had errors while parsing
Errorneous samples [300, 1182, 1488]


In [9]:
# Renaming the columns to singnify what they mean helps with documentation,
# and also helps you keep track of them later on.
train_features.columns = ['feature','label']
# train_features.head()

In [10]:
# this library helps us convert string labels into easy to handle encoded labels.
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
X = np.array(train_features.feature.tolist())
Y = np.array(train_features.label.tolist())
lb = LabelEncoder()
# Since labels are categories they dont inherently have an order amongst themselves.
# For example, Apples > oranges does not make any sense. So to madel such categorical 
# variables, we can convert them to one hot vectors.
Y = to_categorical(lb.fit_transform(Y))

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [13]:
number_of_labels = Y.shape[1]
filter_size = 2

In [14]:
model = Sequential()

In [15]:
model.add(Dense(256, input_shape=(6,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [16]:
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [17]:
model.add(Dense(number_of_labels))
model.add(Activation('softmax'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               1792      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
__________

In [19]:
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')


In [20]:
model.fit(X,Y, batch_size=32, epochs=50)

Epoch 1/50
5435/5435 [==============================] - ETA: 1:29 - loss: 2.3026 - acc: 0.062 - ETA: 11s - loss: 2.3015 - acc: 0.113 - ETA: 6s - loss: 2.3002 - acc: 0.1146 - ETA: 4s - loss: 2.2979 - acc: 0.120 - ETA: 3s - loss: 2.2956 - acc: 0.123 - ETA: 2s - loss: 2.2913 - acc: 0.135 - ETA: 2s - loss: 2.2875 - acc: 0.136 - ETA: 1s - loss: 2.2850 - acc: 0.134 - ETA: 1s - loss: 2.2810 - acc: 0.136 - ETA: 1s - loss: 2.2799 - acc: 0.132 - ETA: 1s - loss: 2.2766 - acc: 0.134 - ETA: 1s - loss: 2.2747 - acc: 0.132 - ETA: 1s - loss: 2.2731 - acc: 0.133 - ETA: 1s - loss: 2.2711 - acc: 0.132 - ETA: 0s - loss: 2.2665 - acc: 0.134 - ETA: 0s - loss: 2.2649 - acc: 0.136 - ETA: 0s - loss: 2.2622 - acc: 0.136 - ETA: 0s - loss: 2.2609 - acc: 0.140 - ETA: 0s - loss: 2.2565 - acc: 0.145 - ETA: 0s - loss: 2.2523 - acc: 0.145 - ETA: 0s - loss: 2.2499 - acc: 0.146 - ETA: 0s - loss: 2.2458 - acc: 0.147 - ETA: 0s - loss: 2.2444 - acc: 0.147 - ETA: 0s - loss: 2.2416 - acc: 0.147 - 2s 331us/step - loss: 2.2392

5435/5435 [==============================] - ETA: 2s - loss: 2.1627 - acc: 0.218 - ETA: 5s - loss: 2.1010 - acc: 0.281 - ETA: 2s - loss: 2.0532 - acc: 0.273 - ETA: 1s - loss: 2.0163 - acc: 0.270 - ETA: 1s - loss: 2.0118 - acc: 0.269 - ETA: 1s - loss: 2.0218 - acc: 0.261 - ETA: 1s - loss: 2.0236 - acc: 0.255 - ETA: 1s - loss: 2.0186 - acc: 0.255 - ETA: 1s - loss: 2.0020 - acc: 0.263 - ETA: 1s - loss: 2.0060 - acc: 0.263 - ETA: 1s - loss: 2.0036 - acc: 0.265 - ETA: 1s - loss: 1.9993 - acc: 0.271 - ETA: 1s - loss: 1.9979 - acc: 0.273 - ETA: 1s - loss: 2.0007 - acc: 0.273 - ETA: 0s - loss: 2.0048 - acc: 0.268 - ETA: 0s - loss: 2.0017 - acc: 0.271 - ETA: 0s - loss: 2.0044 - acc: 0.272 - ETA: 0s - loss: 2.0046 - acc: 0.275 - ETA: 0s - loss: 2.0090 - acc: 0.274 - ETA: 0s - loss: 2.0095 - acc: 0.271 - ETA: 0s - loss: 2.0109 - acc: 0.269 - ETA: 0s - loss: 2.0095 - acc: 0.268 - ETA: 0s - loss: 2.0083 - acc: 0.269 - ETA: 0s - loss: 2.0082 - acc: 0.269 - ETA: 0s - loss: 2.0081 - acc: 0.269 - ETA: 

5435/5435 [==============================] - ETA: 1s - loss: 2.1921 - acc: 0.218 - ETA: 1s - loss: 2.0220 - acc: 0.246 - ETA: 1s - loss: 2.0220 - acc: 0.260 - ETA: 1s - loss: 2.0066 - acc: 0.270 - ETA: 1s - loss: 1.9907 - acc: 0.281 - ETA: 0s - loss: 1.9754 - acc: 0.285 - ETA: 0s - loss: 1.9716 - acc: 0.283 - ETA: 0s - loss: 1.9683 - acc: 0.286 - ETA: 0s - loss: 1.9739 - acc: 0.287 - ETA: 0s - loss: 1.9744 - acc: 0.284 - ETA: 0s - loss: 1.9875 - acc: 0.283 - ETA: 0s - loss: 1.9857 - acc: 0.280 - ETA: 0s - loss: 1.9801 - acc: 0.283 - ETA: 0s - loss: 1.9785 - acc: 0.282 - ETA: 0s - loss: 1.9761 - acc: 0.285 - ETA: 0s - loss: 1.9762 - acc: 0.287 - ETA: 0s - loss: 1.9744 - acc: 0.287 - ETA: 0s - loss: 1.9729 - acc: 0.288 - ETA: 0s - loss: 1.9728 - acc: 0.288 - ETA: 0s - loss: 1.9705 - acc: 0.289 - ETA: 0s - loss: 1.9685 - acc: 0.289 - 1s 207us/step - loss: 1.9696 - acc: 0.2878
Epoch 17/50
5435/5435 [==============================] - ETA: 1s - loss: 2.0217 - acc: 0.250 - ETA: 0s - loss: 1.9

5435/5435 [==============================] - ETA: 1s - loss: 2.1657 - acc: 0.250 - ETA: 0s - loss: 1.8681 - acc: 0.315 - ETA: 0s - loss: 1.9027 - acc: 0.300 - ETA: 0s - loss: 1.9088 - acc: 0.294 - ETA: 0s - loss: 1.9308 - acc: 0.288 - ETA: 0s - loss: 1.9295 - acc: 0.289 - ETA: 0s - loss: 1.9263 - acc: 0.299 - ETA: 0s - loss: 1.9244 - acc: 0.297 - ETA: 0s - loss: 1.9295 - acc: 0.293 - ETA: 0s - loss: 1.9317 - acc: 0.294 - ETA: 0s - loss: 1.9315 - acc: 0.294 - ETA: 0s - loss: 1.9299 - acc: 0.295 - ETA: 0s - loss: 1.9225 - acc: 0.299 - ETA: 0s - loss: 1.9291 - acc: 0.297 - ETA: 0s - loss: 1.9298 - acc: 0.298 - ETA: 0s - loss: 1.9291 - acc: 0.300 - ETA: 0s - loss: 1.9291 - acc: 0.303 - ETA: 0s - loss: 1.9308 - acc: 0.303 - ETA: 0s - loss: 1.9329 - acc: 0.302 - 1s 185us/step - loss: 1.9328 - acc: 0.3029
Epoch 25/50
5435/5435 [==============================] - ETA: 1s - loss: 1.9489 - acc: 0.343 - ETA: 0s - loss: 1.9298 - acc: 0.340 - ETA: 0s - loss: 1.9425 - acc: 0.317 - ETA: 0s - loss: 1.9

5435/5435 [==============================] - ETA: 1s - loss: 1.7825 - acc: 0.250 - ETA: 0s - loss: 1.8470 - acc: 0.335 - ETA: 0s - loss: 1.8373 - acc: 0.343 - ETA: 0s - loss: 1.8519 - acc: 0.339 - ETA: 0s - loss: 1.8639 - acc: 0.322 - ETA: 0s - loss: 1.8637 - acc: 0.322 - ETA: 0s - loss: 1.8652 - acc: 0.326 - ETA: 0s - loss: 1.8781 - acc: 0.320 - ETA: 0s - loss: 1.8823 - acc: 0.319 - ETA: 0s - loss: 1.8926 - acc: 0.316 - ETA: 0s - loss: 1.8993 - acc: 0.312 - ETA: 0s - loss: 1.9043 - acc: 0.311 - ETA: 0s - loss: 1.9077 - acc: 0.311 - ETA: 0s - loss: 1.9107 - acc: 0.311 - ETA: 0s - loss: 1.9097 - acc: 0.314 - ETA: 0s - loss: 1.9071 - acc: 0.314 - ETA: 0s - loss: 1.9029 - acc: 0.314 - ETA: 0s - loss: 1.9007 - acc: 0.314 - ETA: 0s - loss: 1.9012 - acc: 0.310 - ETA: 0s - loss: 1.9030 - acc: 0.309 - ETA: 0s - loss: 1.9007 - acc: 0.311 - ETA: 0s - loss: 1.8995 - acc: 0.311 - 1s 210us/step - loss: 1.8993 - acc: 0.3109
Epoch 33/50
5435/5435 [==============================] - ETA: 1s - loss: 1.7

5435/5435 [==============================] - ETA: 1s - loss: 1.8422 - acc: 0.281 - ETA: 0s - loss: 1.8228 - acc: 0.362 - ETA: 0s - loss: 1.8454 - acc: 0.354 - ETA: 0s - loss: 1.8688 - acc: 0.336 - ETA: 0s - loss: 1.8839 - acc: 0.329 - ETA: 0s - loss: 1.8864 - acc: 0.325 - ETA: 0s - loss: 1.8852 - acc: 0.324 - ETA: 0s - loss: 1.8808 - acc: 0.324 - ETA: 0s - loss: 1.8802 - acc: 0.328 - ETA: 0s - loss: 1.8805 - acc: 0.320 - ETA: 0s - loss: 1.8798 - acc: 0.323 - ETA: 0s - loss: 1.8796 - acc: 0.322 - ETA: 0s - loss: 1.8803 - acc: 0.321 - ETA: 0s - loss: 1.8782 - acc: 0.322 - ETA: 0s - loss: 1.8795 - acc: 0.320 - ETA: 0s - loss: 1.8765 - acc: 0.321 - ETA: 0s - loss: 1.8770 - acc: 0.320 - ETA: 0s - loss: 1.8745 - acc: 0.320 - ETA: 0s - loss: 1.8733 - acc: 0.320 - ETA: 0s - loss: 1.8699 - acc: 0.322 - ETA: 0s - loss: 1.8676 - acc: 0.324 - ETA: 0s - loss: 1.8672 - acc: 0.324 - 1s 205us/step - loss: 1.8654 - acc: 0.3257
Epoch 42/50
5435/5435 [==============================] - ETA: 1s - loss: 1.6

5435/5435 [==============================] - ETA: 1s - loss: 1.8123 - acc: 0.343 - ETA: 0s - loss: 1.8603 - acc: 0.346 - ETA: 0s - loss: 1.8534 - acc: 0.335 - ETA: 0s - loss: 1.8121 - acc: 0.354 - ETA: 0s - loss: 1.8084 - acc: 0.359 - ETA: 0s - loss: 1.8084 - acc: 0.354 - ETA: 0s - loss: 1.8164 - acc: 0.352 - ETA: 0s - loss: 1.8189 - acc: 0.347 - ETA: 0s - loss: 1.8228 - acc: 0.349 - ETA: 0s - loss: 1.8186 - acc: 0.351 - ETA: 0s - loss: 1.8295 - acc: 0.347 - ETA: 0s - loss: 1.8392 - acc: 0.343 - ETA: 0s - loss: 1.8391 - acc: 0.344 - ETA: 0s - loss: 1.8440 - acc: 0.339 - ETA: 0s - loss: 1.8472 - acc: 0.338 - ETA: 0s - loss: 1.8457 - acc: 0.337 - ETA: 0s - loss: 1.8473 - acc: 0.339 - ETA: 0s - loss: 1.8502 - acc: 0.338 - ETA: 0s - loss: 1.8510 - acc: 0.338 - 1s 176us/step - loss: 1.8508 - acc: 0.3384


In [21]:
def test_parser(row):
    global test_error_count
    global test_error_labels
    path_to_wav_files = PATH_TO_TEST_AUDIO_FILES
    file_path = path_to_wav_files + str(row.ID) + ".wav"
    try:
        data, sampling_rate = librosa.load(file_path)
        stft = np.abs(librosa.stft(data))
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(data),sr=sampling_rate).T,axis=0)
    except Exception as ex:
        test_error_count += 1
        test_error_labels.append(row.ID)
        return pd.Series([0]*6)
    features = tonnetz
    return pd.Series(features)

In [22]:
test_features = test.progress_apply(test_parser,axis=1, reduce = True)
print("%d samples had errors while parsing" % test_error_count)
print("Errorneous samples", test_error_labels)
save_as_pickle(data=train_features,pickle_file=PATH_TO_PICKLE + SUBMISSION_TITLE + " test.pickle" )

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')



0 samples had errors while parsing
Errorneous samples []


In [23]:
X_test = test_features
# X_test

In [24]:
test_labels = model.predict(X_test, batch_size=32)

In [25]:
test_labels_strings = lb.inverse_transform(test_labels.argmax(axis=1))
# test_labels_strings

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
test['Class'] = test_labels_strings

In [27]:
test.to_csv(PATH_TO_SUBMISSION + SUBMISSION_TITLE + ".csv",index=None)

This approach gives 56% accuracy with the above setup.